<a href="https://colab.research.google.com/github/bspoloo/SIS420-012024/blob/main/Laboratorios/Laboratorio%205/Laboratorio_5_(Usando_Pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos la librerias necesarias:

In [4]:
# Used for directory and path manipulation
import os
# Scientific and vector computation for Python
import numpy as np
# Library for graphics
from matplotlib import pyplot as plt
#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split
import pandas as pd

import torch
from torch import optim  # Para optimizadores como SGD, Adam, etc.
from torch import nn  # Todos los módulos de redes neuronales
from torch.utils.data import DataLoader  # Facilita la gestión de conjuntos de datos creando mini lotes, etc.

import torchvision  # Paquete torch para cosas relacionadas con la visión
import torchvision.transforms as transforms  # Transformaciones que podemos realizar en nuestro conjunto de datos para aumentar
import torch.nn.functional as F  # Funciones sin parámetros, como (algunas) funciones de activación
import torchvision.datasets as datasets  # Conjuntos de datos estándar
from tqdm import tqdm  # ¡Para una barra de progreso agradable!
from torch.utils.data import Dataset

# Le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

## Red neuronal (Pytorch)

### Cargamos los datos del dataset


In [5]:
dataset = pd.read_csv('BirdSpecies_dataset.csv', delimiter=',')

# dataset = pd.read_csv('csgo_round_snapshots.csv', delimiter=',')

#para usarlo a travez de google colab
# dataset = dataset.sample(n=20000, random_state=42)

dataset

,BodyLength,Wingspan,Weight,PlumageColorHead,PlumageColorBack,PlumageColorBelly,PlumageColorWings,PlumageColorTail,BeakLength,BeakWidth,...,SeedDispersalRole,InterspecificInteraction,IntraspecificInteraction,LearningAbility,ToolUse,ParentalCareLevel,BathingBehavior,GroomingBehavior,FlightSpeed,SpeciesClass
0,5.210,9.746,5.815,8.831,1.580,2.257,6.399,3.746,7.541,4.286,...,8.122,7.139,3.319,0.045,4.456,7.167,7.826,7.316,4.245,4
1,2.275,2.496,6.104,1.692,0.856,2.056,0.110,1.277,8.491,5.665,...,6.354,7.772,6.739,2.824,5.807,2.852,3.478,6.132,4.471,4
2,5.927,1.357,5.936,9.536,1.820,7.886,8.658,4.401,5.786,1.107,...,2.389,3.796,3.149,5.075,4.251,4.682,0.966,9.572,9.364,5
3,6.022,6.402,3.772,2.884,8.638,4.623,2.129,1.352,9.475,8.545,...,8.634,8.433,0.997,6.761,4.675,4.749,6.081,6.774,4.162,3
4,7.816,8.374,0.155,4.262,8.726,2.452,1.046,7.110,5.689,0.559,...,8.851,3.259,3.426,0.555,6.317,2.686,8.380,9.903,3.365,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1.254,7.149,2.825,7.918,9.555,0.723,3.481,6.203,4.350,9.535,...,3.927,7.693,8.008,4.772,4.679,8.625,3.468,5.593,1.138,1
24996,0.839,2.453,3.707,4.084,1.251,6.908,3.673,7.989,7.169,8.058,...,7.228,8.552,5.511,4.374,7.875,2.748,5.159,5.037,6.991,0
24997,5.660,3.032,2.401,6.968,2.762,3.947,8.953,3.312,9.513,3.842,...,2.864,0.291,8.553,2.014,6.728,6.666,3.479,4.143,6.778,1
24998,3.230,4.500,6.235,2.701,2.794,8.908,6.133,3.766,8.113,0.095,...,1.340,5.766,7.622,9.077,3.040,6.313,7.400,1.990,2.240,2


### Analisis del dataset

para tener una mejor vista del los tipos de datos del dataset, hacemos un `info()`:

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   BodyLength                25000 non-null  float64
 1   Wingspan                  25000 non-null  float64
 2   Weight                    25000 non-null  float64
 3   PlumageColorHead          25000 non-null  float64
 4   PlumageColorBack          25000 non-null  float64
 5   PlumageColorBelly         25000 non-null  float64
 6   PlumageColorWings         25000 non-null  float64
 7   PlumageColorTail          25000 non-null  float64
 8   BeakLength                25000 non-null  float64
 9   BeakWidth                 25000 non-null  float64
 10  BeakShape                 25000 non-null  float64
 11  BeakColor                 25000 non-null  float64
 12  LegLength                 25000 non-null  float64
 13  LegColor                  25000 non-null  float64
 14  FootTy

Se puede observar que hay columnas con datos que no son numeros, por lo tanto se procede a tratarlos:

In [7]:
#dropeamos la columna de url
dataset = dataset.drop(['ColorVision','NocturnalVision'], axis=1)

dataset

,BodyLength,Wingspan,Weight,PlumageColorHead,PlumageColorBack,PlumageColorBelly,PlumageColorWings,PlumageColorTail,BeakLength,BeakWidth,...,SeedDispersalRole,InterspecificInteraction,IntraspecificInteraction,LearningAbility,ToolUse,ParentalCareLevel,BathingBehavior,GroomingBehavior,FlightSpeed,SpeciesClass
0,5.210,9.746,5.815,8.831,1.580,2.257,6.399,3.746,7.541,4.286,...,8.122,7.139,3.319,0.045,4.456,7.167,7.826,7.316,4.245,4
1,2.275,2.496,6.104,1.692,0.856,2.056,0.110,1.277,8.491,5.665,...,6.354,7.772,6.739,2.824,5.807,2.852,3.478,6.132,4.471,4
2,5.927,1.357,5.936,9.536,1.820,7.886,8.658,4.401,5.786,1.107,...,2.389,3.796,3.149,5.075,4.251,4.682,0.966,9.572,9.364,5
3,6.022,6.402,3.772,2.884,8.638,4.623,2.129,1.352,9.475,8.545,...,8.634,8.433,0.997,6.761,4.675,4.749,6.081,6.774,4.162,3
4,7.816,8.374,0.155,4.262,8.726,2.452,1.046,7.110,5.689,0.559,...,8.851,3.259,3.426,0.555,6.317,2.686,8.380,9.903,3.365,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1.254,7.149,2.825,7.918,9.555,0.723,3.481,6.203,4.350,9.535,...,3.927,7.693,8.008,4.772,4.679,8.625,3.468,5.593,1.138,1
24996,0.839,2.453,3.707,4.084,1.251,6.908,3.673,7.989,7.169,8.058,...,7.228,8.552,5.511,4.374,7.875,2.748,5.159,5.037,6.991,0
24997,5.660,3.032,2.401,6.968,2.762,3.947,8.953,3.312,9.513,3.842,...,2.864,0.291,8.553,2.014,6.728,6.666,3.479,4.143,6.778,1
24998,3.230,4.500,6.235,2.701,2.794,8.908,6.133,3.766,8.113,0.095,...,1.340,5.766,7.622,9.077,3.040,6.313,7.400,1.990,2.240,2


Mostramos el dataset ya con todos los valores numericos:


### Separamos los datos de nuestra columna Y para cada clase:

In [8]:
# hacemos uso del DataFrame llamado 'dataset' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'round_winner')

#creamos una variable temporal que contentra toda la columna de 'round_winner'
y_temp = dataset['SpeciesClass']

# Para la clase 0

#donde y_temp es igual a 0, separamos los datos en train_class_0, test_class_0
data_class_0 = dataset[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1

#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_1 = dataset[y_temp == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

In [9]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_1.shape[0]} donde el 80% es: {train_class_1.shape[0]} y el 20% es: {test_class_1.shape[0]}")
print(f"La cantidad total de datos es: {dataset.shape[0]}")

Para la clase 0 tenemos una cantidad de: 4147 donde el 80% es: 3317 y el 20% es: 830
Para la clase 1 tenemos una cantidad de: 4167 donde el 80% es: 3333 y el 20% es: 834
La cantidad total de datos es: 25000


### Separacion del 80% de los datos para entrenamiento y 20% para pruebas



In [10]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
train_dataset = pd.concat([train_class_0, train_class_1]).values
test_dataset = pd.concat([test_class_0, test_class_1]).values


indices_train = np.random.permutation(len(train_dataset))
indices_test = np.random.permutation(len(test_dataset))

train_dataset = pd.concat([train_class_0, train_class_1]).values

train_dataset = train_dataset[indices_train]
train_test = test_dataset[indices_test]

m_train = len(train_dataset)
m_test = len(test_dataset)

In [11]:
print(m_train)
print(m_test)
print("El total de ejemplos es: ", (m_train + m_test))

6650
1664
El total de ejemplos es:  8314


#### Convertimos los datos a tensores de Pythorch

Este código conviertimos los datos **train_dataset** y **Test_dataset** en tensores de PyTorch.

Creamos la clase  indicar a PyTorch cuáles son los datos de entrada (features) y cuáles son las etiquetas (targets), creamnos una clase personalizada que herede de torch.utils.data.Dataset y sobrescribir los métodos **__len__** y **__getitem__.**

In [12]:
class TuConjuntoDatos(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data[idx, :-1], dtype=torch.float32)  # Todas las columnas excepto la última
        target = torch.tensor(self.data[idx, -1], dtype=torch.long)  # Última columna (etiqueta)
        return features, target

    def get_features_shape(self):
        return (self.data.shape[1] - 1)

hacemos uso de la clase anterior, indicando cual es nuestra columna "target", en este caso la ultima, **-1**:

In [13]:
train_dataset = TuConjuntoDatos(train_dataset, -1)  # No se necesitan etiquetas durante el entrenamiento
test_dataset = TuConjuntoDatos(test_dataset, -1)

In [14]:
print(f"el 80% de datos es:{len(train_dataset)} y el 20% es: {len(test_dataset)}")
print( "la totalidad de datos es: ", dataset.shape[0])

el 80% de datos es:6650 y el 20% es: 1664
la totalidad de datos es:  25000


### Creacion de la clase de Red neuronal MLS

Aquí creamos nuestra red neuronal simple. Para más detalles aquí estamos subclasificando y
Heredando de **nn.Module**.

In [15]:
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

### Definimos la arquitectura del modelo

En este apartado definiremos la cantidad de capas que tendra el modelo:

- **Capa de entrada:** Es la cantidad de caracteristicas con las que cuenta el dataset.
- **Capa de salida:** como este modelo cuenta con dos clases, por lo tanto solo tendra un valor de 2

In [16]:
# Configurando parametros necesario
input_layer_size = train_dataset.get_features_shape() # Entrada de 96 caracteristicas, es la cantidad de columnas existente en el dataset
num_labels = 6         # como es una clasificacion tiene solo 2 salidas

print("la cantidad de elementos en la capa de entrada es: ",input_layer_size)
print("la cantidad de elementos en la capa de salida es: ",num_labels)

la cantidad de elementos en la capa de entrada es:  58
la cantidad de elementos en la capa de salida es:  6


#### Hiperparametros:

Hiperparámetros de nuestra red neuronal que dependen del conjunto de datos, y
También simplemente experimentando para ver qué funciona bien (tasa de aprendizaje, por ejemplo).

In [17]:
input_size = input_layer_size
num_classes = num_labels
learning_rate = 0.001
batch_size = 1000
num_epochs = 3

print("la cantidad de elementos en la entrada es: ",input_size)

la cantidad de elementos en la entrada es:  58


#### Verificamos que el GPU este habilitado


Este código sirve para configurar el dispositivo en el que se ejecutarán los cálculos de PyTorch. Si una GPU está disponible en el sistema, el código configura el dispositivo para usar la GPU (CUDA), de lo contrario, se configura para utilizar la CPU.

In [19]:
# Configuramos el dispositivo cuda para GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


#### Cargamos conjuntos de datos en lotes:

* **dataset:** El conjunto de datos que se va a cargar en lotes. En este caso, * * * train_dataset y test_dataset son los conjuntos de datos de entrenamiento y prueba, respectivamente.
* **batch_size:** El tamaño de los lotes de datos. Especifica cuántas muestras se cargarán a la vez durante el entrenamiento o la evaluación.
* **shuffle:** Un booleano que indica si se deben mezclar los datos antes de cargarlos en lotes. Esto es útil durante el entrenamiento para evitar que el modelo aprenda patrones basados en el orden de los datos.

In [20]:
print(len(train_dataset))
print(len(test_dataset))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

print("la cantidad de lotes para el dataset de entrenamiento es: ",len(train_loader))
print("la cantidad de lotes para el dataset de prueba es: ",len(test_loader))

6650
1664
la cantidad de lotes para el dataset de entrenamiento es:  7
la cantidad de lotes para el dataset de prueba es:  2


### Modelo de red neuronal

Este código crea una instancia de un modelo de red neuronal y lo mueve al dispositivo especificado (ya sea GPU o CPU) para su entrenamiento y evaluación.

* **RedNeuronalMLS:** Usamos la clase para crear el modelo.
* **input_size:** es el tamaño de entrada del modelo, es decir, el número de características en cada ejemplo de entrada.
num_classes es el número de clases en el problema de clasificación, es decir, el número de categorías diferentes a predecir.
* **.to(device):** mueve el modelo a la GPU (cuda) si está disponible, de lo contrario, lo deja en la CPU (cpu).

In [21]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

Mostramos algunas caracteristicas del modelo:

In [22]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=58, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

#### Funcion de perdida o costo para el modelo:

como este es un modelo de clasificacion, se procede a crearlo con la funcion de **CrossEntropyLoss()** y el optimizador creando una instancia del **optimizador Adam**.

In [23]:
# Perdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Entrenamiento de la Red neuronal

Haciendo el uso dos ciclos For, en el primero para el uso del los **"epochs"** el cual ira iterando uno por uno dentro del segundo For, el c se recorre el cargador de datos de entrenamiento **(train_loader)** para obtener lotes **(batches)** de datos y sus respectivas etiquetas. **batch_idx **es el índice del lote actual, data contiene las entradas del lote y targets contiene las etiquetas correspondientes.

In [24]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):

        # Obtenemos el CUDA si es posible
        data = data.to(device=device)
        targets = targets.to(device=device)


        # Llegar a la forma correcta
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

 14%|█▍        | 1/7 [00:00<00:01,  4.99it/s]

torch.Size([1000, 58])


100%|██████████| 7/7 [00:00<00:00, 25.25it/s]


torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([650, 58])


100%|██████████| 7/7 [00:00<00:00, 76.85it/s]


torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([650, 58])


  0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([1000, 58])
torch.Size([650, 58])


100%|██████████| 7/7 [00:00<00:00, 68.09it/s]


### validaciones

Para hacer las validaciones correspondientes, se **creo check_accuracy()** el cual recibe como parametros nuestros dataset_loader y el modelo, donde nos retornara el procentaje de precision y las predicciones hechas en un vector:

In [25]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

#### Precision con los datos de entrenamiento:

In [26]:
p_train, pred_train  = check_accuracy(train_loader, model)
print(f"Precisión en el conjunto de entrenamiento: {p_train*100:.2f} %")

Precisión en el conjunto de entrenamiento: 49.25 %


#### Precision con los datos de prueba:

In [27]:
p_test, pred_test  = check_accuracy(test_loader, model)
print(f"Precisión en el conjunto de prueba: {p_test*100:.2f} %")

Precisión en el conjunto de prueba: 48.86 %


## ¿Concluciones?

Una vez hecha las predicciones del modelos, se puede determinar que el modelo reliza las predicciones con una precision del 49.25% con los datos de prueba y con un 48.86% con los datos de entrenamiento, entonces podemos concluir que no es una precision buena.